In [1]:
import torch
import torch.nn as nn

In [2]:
## Creating a residual block

class ResidualBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels, use_1x1_conv=False,
                 stride=1):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                              stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                              stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.relu = nn.ReLU(inplace=True)
                
        if use_1x1_conv:
            self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=1,
                                  stride=stride)
        else:
            self.conv3 = None
            
    def forward(self, x):
        y = self.relu(self.bn1(self.conv1(x)))
        y = self.bn2(self.conv2(y))

        
        if self.conv3:
            ## dimensions will be matched
            x = self.conv3(x)
            
        ## returns f(x) + x
        return self.relu(x+y)
        
        
        



In [5]:
output_size=133
class ResNet18(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64,
                              kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self.get_renset_layer(64, 64, 2, first_block=True)
        self.layer2 = self.get_renset_layer(64, 128, 2)
        self.layer3 = self.get_renset_layer(128, 256, 2)
        self.layer4 = self.get_renset_layer(256, 512, 2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512, output_size)
        
        
    
    def get_renset_layer(self, in_channels, out_channels,
                         num_residual_blocks, first_block=False):
        
        layers = list()
        
        for i in range(num_residual_blocks):
            if i == 0 and not first_block:
                layers.append(ResidualBlock(in_channels, out_channels,
                                use_1x1_conv=True, stride=2))
            else:
                layers.append(ResidualBlock(in_channels, out_channels,
                                use_1x1_conv=False, stride=1))
            in_channels = out_channels
            
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        
        return x
        
        

In [6]:
model = ResNet18()
model

ResNet18(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1